### Desafio ENACOM

##### Descrição do problema

Considerando um capital para investimento de R$1.000.000 e as seguintes opções de investimento:

|Opção| Custo do investimento   | Retorno esperado      | 
|:---:|      :----:             |       :---:           |
|   1 | 470.000                 | 410.000               |
|   2 | 400.000                 | 330.000               |
|   3 | 170.000                 | 140.000               |
|   4 | 270.000                 | 250.000               |
|   5 | 340.000                 | 320.000               |
|   6 | 230.000                 | 320.000               |
|   7 |  50.000                 | 90.000                |
|   8 | 440.000                 | 190.000               |


Vamos desenvolver um algoritmo de otimização para selecionar os projetos que maximizam o retorno total esperado, considerando que:

- Se o investimento 1 for selecionado, então o investimento 5 não deve ser;

- Se o investimento 2 for selecionado, então o investimento 4 também deve ser.

##### Classe de dados

In [1]:
from typing import List
import pandas as pd

# Classe de dados

from dataclasses import dataclass


@dataclass
class ValuableItem:
    name: str
    profit: float
    cost: float

    @property
    def value_density(self):
        return self.profit/(self.cost)

##### Métodos auxiliares

In [2]:
def items_to_table(items: List[ValuableItem]) -> pd.DataFrame:
    records = [{
        'Item': i.name,
        'Lucro (R$)': i.profit,
        'Custo (R$)': i.cost
    } for i in items]
    records.append({
        'Item': 'Total',
        'Lucro (R$)': sum(i.profit for i in items),
        'Custo (R$)': sum(i.cost for i in items)
    })
    return pd.DataFrame.from_records(records)

##### Entrada dos dados

In [3]:
max_cost = 1000000

profits = [
    410000, 330000, 140000, 250000, 320000, 320000, 90000, 190000
]

costs = [
    470000, 400000, 170000, 270000, 340000, 230000, 50000, 440000
]

available_items = [ValuableItem(f'Item {i}', v, w)
                   for i, (v, w) in enumerate(zip(profits, costs))]

items_to_table(available_items)


,Item,Lucro (R$),Custo (R$)
0,Item 0,410000,470000
1,Item 1,330000,400000
2,Item 2,140000,170000
3,Item 3,250000,270000
4,Item 4,320000,340000
5,Item 5,320000,230000
6,Item 6,90000,50000
7,Item 7,190000,440000
8,Total,2050000,2370000


##### Heurística gulosa

In [4]:
def greedy(max_cost, available_items):
    chosen_items = []
    sorted_items = sorted(
        available_items,
        key=lambda i: i.value_density,
        reverse=True)
    for item in sorted_items:
        if item.cost <= max_cost:
            chosen_items.append(item)
            max_cost -= item.cost
    return chosen_items

In [5]:
def greedy_with4(max_cost, available_items):
    chosen_items = list(filter(lambda item: item.name == "Item 3", available_items))
if sum(item.cost for item in chosen_items) <= max_cost:
        for item in chosen_items:
            max_cost -= item.cost
            sorted_items = sorted(
                available_items, key=lambda i: i.value_density, reverse=True
            )
            for item in sorted_items:
                if item.cost <= max_cost:
                    chosen_items.append(item)
                    max_cost -= item.cost
    return chosen_items


IndentationError: unexpected indent (2932694346.py, line 3)

In [ ]:
def chosen_itens_contains_2(
    max_cost: float, available_items: List[ValuableItem]
) -> bool:
    for item in greedy(max_cost, available_items):
        if item.name == "Item 1":
            return True
    return False

In [ ]:
def chosen_itens_contains_4(
    max_cost: float, available_items: List[ValuableItem]
) -> bool:
    for item in greedy(max_cost, available_items):
        if item.name == "Item 3":
            return True
    return False

In [ ]:
def chosen_itens_contains_2_after_4(
    max_cost: float, available_items: List[ValuableItem]
) -> bool:
    for item in greedy_with4(max_cost, available_items):
        if item.name == "Item 1":
            return True
    return False

In [ ]:
def greedy_restriction2(
    max_cost: float, available_items: List[ValuableItem]
) -> List[ValuableItem]:
    """Resolve a segunda restrição.

    Args:
        max_cost (float): custo máximo.
        available_items (List[ValuableItem]): retorna a lista já ordenada.

    Returns:
        List[ValuableItem]: retorna a lista já ordenada e com a restrição satisfeita.
    """
    available_items_without2 = list(
        filter(lambda item: item.name != "Item 1", available_items)
    )  # remove o item 2 da lista
    if chosen_itens_contains_2(max_cost, available_items):
        if chosen_itens_contains_4(max_cost, available_items):
            return greedy(max_cost, available_items)
        else:
            if sum(
                item.profit for item in greedy(max_cost, available_items_without2)
            ) >= sum(item.profit for item in greedy_with4(max_cost, available_items)):
                return greedy(max_cost, available_items_without2)
            else:
                if chosen_itens_contains_2_after_4:
                    return greedy_with4(max_cost, available_items)
                else:
                    return greedy(max_cost, available_items_without2)
    else:
        return greedy(max_cost, available_items)

In [ ]:
def greedy_knapsack(max_cost, available_items):
    if available_items[0].value_density >= available_items[4].value_density:
        return greedy_restriction2(max_cost, available_items[:4]+available_items[5:])
    else:
        return greedy_restriction2(max_cost, available_items[1:])

In [ ]:
chosen_items = greedy_knapsack(max_cost, available_items)
items_to_table(chosen_items)

##### Modelagem

**Parâmetros**:

* $c$ - Escalar ($\mathbb R$) representando o capital de investimento
* $w$ - Vetor ($\mathbb R^n$) representand o custo de cada investimento
* $v$ - Vetor ($\mathbb R^n$) representand o lucro de cada investimento

**Variáveis de decisão**

* $x$ - Vetor binário ($\mathbb B^n$) indicando se o $n$-ésimo item foi escolhido ou não

**Formulação do problema**:

$$ \max_x \; v^Tx, \text{where }\ x\in\mathbb B $$
$$ \text{subject to: }$$ 
$$ w^Tx \leq c $$
$$ x_{1} + x_{5}\le1 $$
$$ x_{2} - 2*x_{4}\le 0 $$


In [ ]:
!pip install pyomo
!apt install glpk-utils
!pip install glpk

In [ ]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

In [ ]:
from pyomo.core.expr.current import exp
model = pyo.ConcreteModel()

# Índices

model.M = pyo.RangeSet(1,8)

# Parâmetros

u = {}
u[1] = 410000
u[2] = 330000
u[3] = 140000
u[4] = 250000
u[5] = 320000
u[6] = 320000
u[7] = 90000
u[8] = 190000
model.p = pyo.Param(model.M, initialize=u)

r = {}
r[1] = 470000
r[2] = 400000
r[3] = 170000
r[4] = 270000
r[5] = 340000
r[6] = 230000
r[7] = 50000
r[8] = 440000
model.q = pyo.Param(model.M, initialize=r)

# Variáveis de decisão

model.x = pyo.Var(model.M, within=pyo.Binary)

# Função objetiva

model.obj_func = pyo.Objective(expr = sum(model.p[i]*model.x[i] for i in model.M), sense = pyo.maximize)

# Restrições

model.const_func1 = pyo.Constraint(expr = sum(model.q[i]*model.x[i] for i in model.M) <=1000000)
model.const_func2 = pyo.Constraint(expr=model.x[1]+model.x[5]<=1)
model.const_func3 = pyo.Constraint(expr=model.x[2]-2*model.x[4]<=0)

# Resultados

optm = SolverFactory('glpk')
results = optm.solve(model)
custo = sum(model.q[i]*pyo.value(model.x[i]) for i in model.M)
print("Custo: ",custo)
for i in model.M:
  print("x[{}]={}".format(i,pyo.value(model.x[i])))


print('Lucro esperado: R${}'.format(pyo.value(model.obj_func)))